In [1]:
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import ctypes
import logging
from datetime import date
from random import randint

# Hardcoded variables

In [2]:
url = r'https://sedeclave.dgt.gob.es/WEB_NCIT_CONSULTA/solicitarCita.faces'

In [3]:
oficina = 'Ávila'
#oficina = 'Madrid'
#oficina = 'Badajoz'

trámite = 'Trámites de oficina'

In [4]:
driver_path = Path('chromedriver.exe')

In [5]:
datemark = date.today().strftime("%Y%m%d")

In [6]:
log_path = Path(f'{datemark}_DGT_cita_previa.log')

# Aux functions

In [7]:
def launch_site(driver_path, url):
    driver = webdriver.Chrome(str(driver_path))
    driver.implicitly_wait(3)
    driver.get(url)
    return driver

In [8]:
def fill_office(driver):
    elem = driver.find_element_by_id('publicacionesForm:oficina')
    elem.click()
    time.sleep(randint(1,5))
    elem.send_keys(oficina + Keys.ENTER)

In [9]:
def fill_query(driver):
    elem = driver.find_element_by_id('publicacionesForm:tipoTramite')
    elem.click()
    time.sleep(randint(1,2))
    elem.send_keys(trámite + Keys.ENTER)

In [10]:
def click_continue(driver):
    elem = driver.find_element_by_css_selector('#publicacionesForm\:j_id59 > input')
    elem.click()

In [11]:
def query_office(driver):
    fill_office(driver)
    fill_query(driver)
    time.sleep(randint(1,3))
    click_continue(driver)

In [12]:
def query_until(driver, freq=5, randomize=True):
    query_office(driver)
    time.sleep(5 + randint(0,5))
    
    retry = False
    if fully_booked(driver):
        logging.warning(f'Failed: fully booked')
        retry = True
        
    elif request_overflow(driver):
        logging.warning(f'Failed: request overflow')
        retry = True
    
    if retry:
        secs = freq * 60
        if randomize:
            secs += randint(0, secs)  # add up to double freq (random)
        time.sleep(secs)
        query_until(driver, freq)
    else:
        logging.warning(f'Success: appointment available')
        # bring to front
        driver.minimize_window()  
        driver.maximize_window()
        # message box
        title = 'ALERT'
        text = 'Appointment available'
        ctypes.windll.user32.MessageBoxW(0, text, title, 0x00001000)

In [13]:
def fully_booked(driver):
    try:
        elem = driver.find_element_by_class_name('msgError')
        return elem.text == 'El horario de atención al cliente está completo para los próximos días. Inténtelo más tarde.'
    except NoSuchElementException:
        return False

In [14]:
def request_overflow(driver):
    try:
        elem = driver.find_element_by_class_name('msgError')
        return elem.text == 'Estamos recibiendo un número muy elevado de accesos que no nos permiten procesar tu petición. Por favor, inténtalo de nuevo pasados unos minutos.'
    except NoSuchElementException:
        return False

In [15]:
def available_areas(driver):
    try:
        elems = driver.find_elements_by_xpath("//*[contains(text(), 'Área: ')]")
        return [e.text for e in elems]
    except NoSuchElementException:
        return list()

In [16]:
def areas_buttons(driver):
    try:
        elems = driver.find_elements_by_xpath("//*[contains(@title, 'Continuar')]")
        return elems
    except NoSuchElementException:
        return list()

In [17]:
def areas_dict(driver):
    return dict(zip(available_areas(driver), areas_buttons(driver)))

In [18]:
def launch_logger(log_path, overwrite=False):
    
    # Check
    if log_path.exists(): 
        if overwrite:
            log_path.unlink()
        else:
            raise FileExistsError(log_path)
            
    FORMAT = '%(asctime)s,%(name)s,%(levelname)s,%(message)s'
    logging.basicConfig(filename=log_path, format=FORMAT, level=logging.WARNING)

In [19]:
if __name__ == '__main__':
    
    launch_logger(log_path, overwrite=True)
    driver = launch_site(driver_path, url)
    query_until(driver, freq=5)
    buttons = areas_dict(driver)

In [20]:
buttons

{'Área: Trámites generales': <selenium.webdriver.remote.webelement.WebElement (session="7f0364cbf7a839be8e1b85f36fe1e368", element="6531740a-e5a1-4a89-9bc8-fda551c23970")>}

In [21]:
#driver.close()
logging.shutdown()